In [1]:
%matplotlib inline

# Transfer Learning
In this assignment, we will use the weights of a network pre-trained in a particular problem as starting point to train our CNN to a different problem. As training a network from scratch is time-consuming and demands a lot of data, this is a frequent strategy, specially if both datasets (the one used for pre-training and the target) shares similar structures/elements/concepts. 

This is specially true when working with images. Most filters learned in initial convolutional layers will detect low-level elements, such as borders, corners and color blobs, which are common to most problems in the image domain. 

In this notebook, we will load the SqueezeNet architecture trained in the ImageNet dataset and fine-tune it to CIFAR-10.

## Imports

In [2]:
import os
import numpy as np
from time import time
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array
from keras.datasets import cifar10
from keras.callbacks import TensorBoard
from sklearn.model_selection import StratifiedShuffleSplit

#Utility to plot
def plotImages(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])
        
        
def plotImage(img):
    fig = plt.figure(figsize=(3,3))
    ax = fig.add_subplot(111)

    ax.imshow(np.uint8(img), interpolation='nearest')
    plt.show()

Using TensorFlow backend.


## SqueezeNet definition
These methods define our architecture and load the weights obtained using ImageNet data.

In [3]:
# Fire Module Definition
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

#SqueezeNet model definition
def SqueezeNet(input_shape):
    img_input = Input(shape=input_shape) #placeholder
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)

    model = Model(img_input, x, name='squeezenet')

    # Download and load ImageNet weights
    model.load_weights('./squeezenet_weights_tf_dim_ordering_tf_kernels.h5')
    
    return model    

## CIFAR-10

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. The class are **airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck**.

In [4]:
#Load data
(trainVal_data, trainVal_label), (X_test, y_test) = cifar10.load_data()
print("Train/Val data. X: ", trainVal_data.shape, ", Y: ", trainVal_data.shape)
print("Test data. X: ", X_test.shape, ", Y: ", y_test.shape)

Train/Val data. X:  (50000, 32, 32, 3) , Y:  (50000, 32, 32, 3)
Test data. X:  (10000, 32, 32, 3) , Y:  (10000, 1)


In [5]:
# Prepare the data 
Y = np.zeros((len(trainVal_label),10))
for i in range(len(trainVal_label)):
    Y[i][trainVal_label[i][0]] = 1

y = np.zeros((len(y_test),10))
for i in range(len(y_test)):
    y[i][y_test[i][0]] = 1
    
y_test = y
trainVal_label = Y

In [6]:
x_train = trainVal_data[:40000]
y_train = trainVal_label[:40000]
x_valid = trainVal_data[40000:]
y_valid = trainVal_label[40000:]

-----------------
## SqueezeNet with frozen layers
Our initial attempt will be to remove SqueezeNet's top layers --- responsible for the classification into ImageNet classes --- and train a new set of layers to our CIFAR-10 classes. We will also freeze the layers before `drop9`. Our architecture will be like this:

<img src="frozenSqueezeNet.png" width=70% height=70%>

In [7]:
squeezeNetModel = SqueezeNet((32,32,3))

#freeze layers
for layer in squeezeNetModel.layers:
    layer.trainable = False

#squeezeNetModel.summary()
    
#Add new classification layers
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()

#squeezeNetModel.summary()

x = squeezeNetModel.layers[-1].output
x = Convolution2D(10, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 15, 15, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 15, 15, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 7, 7, 64)     0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

Now, we compile our model and train it:

In [8]:
# Compile model and train it.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tbCallBack = TensorBoard(log_dir="./TransferLearning/last_layers/{}".format(time()), write_graph=True)

model.fit(x=x_train, y=y_train, batch_size=50, epochs=500, verbose=1, callbacks=[tbCallBack], validation_split=0, 
        validation_data=(x_valid, y_valid), shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, 
        steps_per_epoch=None, validation_steps=None)

model.save('last_layers.h5')

Train on 40000 samples, validate on 10000 samples
Epoch 1/500
40000/40000 [==============================] - 16s 395us/step - loss: 2.9477 - acc: 0.1154 - val_loss: 2.2979 - val_acc: 0.1029
Epoch 2/500
40000/40000 [==============================] - 14s 360us/step - loss: 2.2897 - acc: 0.1214 - val_loss: 2.2861 - val_acc: 0.1141
Epoch 3/500
40000/40000 [==============================] - 13s 335us/step - loss: 2.2638 - acc: 0.1417 - val_loss: 2.2256 - val_acc: 0.1443
Epoch 4/500
40000/40000 [==============================] - 13s 334us/step - loss: 2.2241 - acc: 0.1670 - val_loss: 2.1616 - val_acc: 0.1888
Epoch 5/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1872 - acc: 0.1896 - val_loss: 2.1144 - val_acc: 0.2163
Epoch 6/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1668 - acc: 0.2069 - val_loss: 2.1156 - val_acc: 0.2408
Epoch 7/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.1532 - acc: 0.2157 - val_l

Epoch 59/500
40000/40000 [==============================] - 13s 330us/step - loss: 2.1326 - acc: 0.2345 - val_loss: 2.0783 - val_acc: 0.2483
Epoch 60/500
40000/40000 [==============================] - 13s 329us/step - loss: 2.1339 - acc: 0.2375 - val_loss: 2.0914 - val_acc: 0.2413
Epoch 61/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1302 - acc: 0.2403 - val_loss: 2.0744 - val_acc: 0.2570
Epoch 62/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1338 - acc: 0.2395 - val_loss: 2.0966 - val_acc: 0.2443
Epoch 63/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1286 - acc: 0.2382 - val_loss: 2.0875 - val_acc: 0.2474
Epoch 64/500
40000/40000 [==============================] - 13s 330us/step - loss: 2.1319 - acc: 0.2378 - val_loss: 2.0893 - val_acc: 0.2487
Epoch 65/500
40000/40000 [==============================] - 13s 330us/step - loss: 2.1269 - acc: 0.2377 - val_loss: 2.0870 - val_acc: 0.2476
Epoch 66/500


Epoch 117/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.1169 - acc: 0.2473 - val_loss: 2.0649 - val_acc: 0.2595
Epoch 118/500
40000/40000 [==============================] - 13s 330us/step - loss: 2.1182 - acc: 0.2464 - val_loss: 2.0557 - val_acc: 0.2668
Epoch 119/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.1232 - acc: 0.2429 - val_loss: 2.1169 - val_acc: 0.2200
Epoch 120/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1204 - acc: 0.2497 - val_loss: 2.0699 - val_acc: 0.2555
Epoch 121/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.1167 - acc: 0.2507 - val_loss: 2.0514 - val_acc: 0.2699
Epoch 122/500
40000/40000 [==============================] - 13s 334us/step - loss: 2.1199 - acc: 0.2465 - val_loss: 2.0567 - val_acc: 0.2653
Epoch 123/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.1187 - acc: 0.2476 - val_loss: 2.0643 - val_acc: 0.2697
Epoch 

Epoch 175/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0924 - acc: 0.2546 - val_loss: 2.0294 - val_acc: 0.2824
Epoch 176/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0950 - acc: 0.2553 - val_loss: 2.0221 - val_acc: 0.2754
Epoch 177/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0970 - acc: 0.2572 - val_loss: 2.0587 - val_acc: 0.2654
Epoch 178/500
40000/40000 [==============================] - 13s 336us/step - loss: 2.0939 - acc: 0.2531 - val_loss: 2.0508 - val_acc: 0.2574
Epoch 179/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.0949 - acc: 0.2534 - val_loss: 2.0292 - val_acc: 0.2777
Epoch 180/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.1008 - acc: 0.2511 - val_loss: 2.0248 - val_acc: 0.2687
Epoch 181/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0992 - acc: 0.2520 - val_loss: 2.0201 - val_acc: 0.2807
Epoch 

Epoch 233/500
40000/40000 [==============================] - 13s 335us/step - loss: 2.0971 - acc: 0.2531 - val_loss: 2.0374 - val_acc: 0.2671
Epoch 234/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0942 - acc: 0.2584 - val_loss: 2.0295 - val_acc: 0.2692
Epoch 235/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.0973 - acc: 0.2534 - val_loss: 2.0321 - val_acc: 0.2832
Epoch 236/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0966 - acc: 0.2552 - val_loss: 2.0386 - val_acc: 0.2754
Epoch 237/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0907 - acc: 0.2575 - val_loss: 2.0298 - val_acc: 0.2693
Epoch 238/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0980 - acc: 0.2535 - val_loss: 2.0425 - val_acc: 0.2626
Epoch 239/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0961 - acc: 0.2549 - val_loss: 2.0197 - val_acc: 0.2824
Epoch 

Epoch 291/500
40000/40000 [==============================] - 14s 342us/step - loss: 2.1002 - acc: 0.2523 - val_loss: 2.0456 - val_acc: 0.2637
Epoch 292/500
40000/40000 [==============================] - 13s 335us/step - loss: 2.0967 - acc: 0.2512 - val_loss: 2.0362 - val_acc: 0.2706
Epoch 293/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0936 - acc: 0.2533 - val_loss: 2.0350 - val_acc: 0.2731
Epoch 294/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.1021 - acc: 0.2537 - val_loss: 2.0399 - val_acc: 0.2680
Epoch 295/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0942 - acc: 0.2513 - val_loss: 2.0202 - val_acc: 0.2811
Epoch 296/500
40000/40000 [==============================] - 13s 336us/step - loss: 2.0966 - acc: 0.2539 - val_loss: 2.0252 - val_acc: 0.2764
Epoch 297/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0953 - acc: 0.2546 - val_loss: 2.0362 - val_acc: 0.2701
Epoch 

Epoch 349/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0936 - acc: 0.2543 - val_loss: 2.0202 - val_acc: 0.2731
Epoch 350/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.0995 - acc: 0.2530 - val_loss: 2.0367 - val_acc: 0.2718
Epoch 351/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.1003 - acc: 0.2532 - val_loss: 2.0289 - val_acc: 0.2771
Epoch 352/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.1006 - acc: 0.2498 - val_loss: 2.0270 - val_acc: 0.2723
Epoch 353/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0952 - acc: 0.2567 - val_loss: 2.0066 - val_acc: 0.2867
Epoch 354/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0969 - acc: 0.2546 - val_loss: 2.0200 - val_acc: 0.2780
Epoch 355/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.0936 - acc: 0.2562 - val_loss: 2.0315 - val_acc: 0.2762
Epoch 

Epoch 407/500
40000/40000 [==============================] - 13s 330us/step - loss: 2.0922 - acc: 0.2532 - val_loss: 2.0480 - val_acc: 0.2573
Epoch 408/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0957 - acc: 0.2510 - val_loss: 2.0260 - val_acc: 0.2708
Epoch 409/500
40000/40000 [==============================] - 13s 334us/step - loss: 2.0933 - acc: 0.2515 - val_loss: 2.0338 - val_acc: 0.2669
Epoch 410/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0986 - acc: 0.2528 - val_loss: 2.0299 - val_acc: 0.2669
Epoch 411/500
40000/40000 [==============================] - 13s 334us/step - loss: 2.0962 - acc: 0.2574 - val_loss: 2.0512 - val_acc: 0.2718
Epoch 412/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0968 - acc: 0.2550 - val_loss: 2.0488 - val_acc: 0.2712
Epoch 413/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.0970 - acc: 0.2540 - val_loss: 2.0288 - val_acc: 0.2675
Epoch 

Epoch 465/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0979 - acc: 0.2512 - val_loss: 2.0479 - val_acc: 0.2568
Epoch 466/500
40000/40000 [==============================] - 13s 333us/step - loss: 2.0970 - acc: 0.2524 - val_loss: 2.0274 - val_acc: 0.2755
Epoch 467/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0935 - acc: 0.2558 - val_loss: 2.0455 - val_acc: 0.2661
Epoch 468/500
40000/40000 [==============================] - 13s 332us/step - loss: 2.0994 - acc: 0.2502 - val_loss: 2.0329 - val_acc: 0.2744
Epoch 469/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0975 - acc: 0.2530 - val_loss: 2.0377 - val_acc: 0.2727
Epoch 470/500
40000/40000 [==============================] - 13s 331us/step - loss: 2.0942 - acc: 0.2536 - val_loss: 2.0366 - val_acc: 0.2722
Epoch 471/500
40000/40000 [==============================] - 14s 343us/step - loss: 2.0971 - acc: 0.2538 - val_loss: 2.0506 - val_acc: 0.2656
Epoch 

Finally, let's evaluate on our test set:

In [9]:
# Evaluate on validation:
score = model.evaluate(x=x_valid, y=y_valid, batch_size=None, verbose=1, sample_weight=None, steps=None)
print('Validation loss:', score[0])
print('Validation accuracy (NORMALIZED):', score[1])

10000/10000 [==============================] - 3s 272us/step
Validation loss: 2.023463514709473
Validation accuracy (NORMALIZED): 0.2772


-----------------
-----------------

# Training last 2 Fire Modules + classification layers
As we could see, the frozen network performed very poorly. By freezing most layers, we do not allow SqueezeNet to adapt its weights to features present in CIFAR-10.

Let's try to unfreeze the last two fire modules and train once more. The architecture will be:
<img src="partFrozenSqueezeNet.png" width=70% height=70%>

In [10]:
squeezeNetModel = SqueezeNet((32,32,3))

layers = [layer.name for layer in squeezeNetModel.layers]


#freeze the mentioned layers
for layer in squeezeNetModel.layers[0:layers.index('fire7/concat')+1]:
    layer.trainable = False

#Add new classification layers
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()

#squeezeNetModel.summary()

x = squeezeNetModel.layers[-1].output
x = Convolution2D(10, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 15, 15, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 15, 15, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 7, 7, 64)     0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

Now, we compile our model and train it:

In [11]:
#Compile model and train it
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tbCallBack = TensorBoard(log_dir="./TransferLearning/fire8_log/{}".format(time()), write_graph=True)

model.fit(x=x_train, y=y_train, batch_size=50, epochs=500, verbose=1, callbacks=[tbCallBack], validation_split=0, 
        validation_data=(x_valid, y_valid), shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, 
        steps_per_epoch=None, validation_steps=None)

model.save('fire8.h5')

Train on 40000 samples, validate on 10000 samples
Epoch 1/500
40000/40000 [==============================] - 23s 585us/step - loss: 2.1893 - acc: 0.1740 - val_loss: 1.9150 - val_acc: 0.2890
Epoch 2/500
40000/40000 [==============================] - 23s 568us/step - loss: 2.0120 - acc: 0.2303 - val_loss: 1.8259 - val_acc: 0.3471
Epoch 3/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.9020 - acc: 0.2926 - val_loss: 1.6784 - val_acc: 0.4037
Epoch 4/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.7964 - acc: 0.3567 - val_loss: 1.6201 - val_acc: 0.4343
Epoch 5/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.7272 - acc: 0.3939 - val_loss: 1.5981 - val_acc: 0.4456
Epoch 6/500
40000/40000 [==============================] - 23s 568us/step - loss: 1.6814 - acc: 0.4086 - val_loss: 1.5829 - val_acc: 0.4456
Epoch 7/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.6303 - acc: 0.4367 - val_l

Epoch 59/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.1639 - acc: 0.5932 - val_loss: 1.6584 - val_acc: 0.4911
Epoch 60/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.1719 - acc: 0.5909 - val_loss: 1.6332 - val_acc: 0.4856
Epoch 61/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.1570 - acc: 0.5958 - val_loss: 1.7455 - val_acc: 0.4786
Epoch 62/500
40000/40000 [==============================] - 23s 574us/step - loss: 1.1443 - acc: 0.5975 - val_loss: 1.7510 - val_acc: 0.4802
Epoch 63/500
40000/40000 [==============================] - 23s 568us/step - loss: 1.1540 - acc: 0.5992 - val_loss: 1.7130 - val_acc: 0.4796
Epoch 64/500
40000/40000 [==============================] - 23s 566us/step - loss: 1.1387 - acc: 0.6010 - val_loss: 1.7690 - val_acc: 0.4823
Epoch 65/500
40000/40000 [==============================] - 23s 571us/step - loss: 1.1518 - acc: 0.5976 - val_loss: 1.7384 - val_acc: 0.4817
Epoch 66/500


Epoch 117/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.1059 - acc: 0.6214 - val_loss: 1.9410 - val_acc: 0.4733
Epoch 118/500
40000/40000 [==============================] - 23s 570us/step - loss: 1.0261 - acc: 0.6463 - val_loss: 1.9484 - val_acc: 0.4649
Epoch 119/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0174 - acc: 0.6486 - val_loss: 1.9985 - val_acc: 0.4708
Epoch 120/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0055 - acc: 0.6537 - val_loss: 1.9821 - val_acc: 0.4742
Epoch 121/500
40000/40000 [==============================] - 23s 568us/step - loss: 1.0212 - acc: 0.6492 - val_loss: 1.9673 - val_acc: 0.4614
Epoch 122/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0070 - acc: 0.6541 - val_loss: 1.9787 - val_acc: 0.4599
Epoch 123/500
40000/40000 [==============================] - 23s 568us/step - loss: 1.0109 - acc: 0.6510 - val_loss: 2.0996 - val_acc: 0.4703
Epoch 

Epoch 175/500
40000/40000 [==============================] - 23s 569us/step - loss: 0.9743 - acc: 0.6690 - val_loss: 2.0785 - val_acc: 0.4632
Epoch 176/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9512 - acc: 0.6748 - val_loss: 2.1375 - val_acc: 0.4644
Epoch 177/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9404 - acc: 0.6770 - val_loss: 2.2673 - val_acc: 0.4624
Epoch 178/500
40000/40000 [==============================] - 23s 566us/step - loss: 0.9519 - acc: 0.6745 - val_loss: 2.2459 - val_acc: 0.4575
Epoch 179/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0103 - acc: 0.6594 - val_loss: 2.1745 - val_acc: 0.4583
Epoch 180/500
40000/40000 [==============================] - 23s 567us/step - loss: 0.9278 - acc: 0.6841 - val_loss: 2.1692 - val_acc: 0.4686
Epoch 181/500
40000/40000 [==============================] - 23s 573us/step - loss: 0.9474 - acc: 0.6756 - val_loss: 2.2938 - val_acc: 0.4567
Epoch 

Epoch 233/500
40000/40000 [==============================] - 23s 573us/step - loss: 0.9137 - acc: 0.6905 - val_loss: 2.4041 - val_acc: 0.4617
Epoch 234/500
40000/40000 [==============================] - 23s 570us/step - loss: 0.9055 - acc: 0.6931 - val_loss: 2.3207 - val_acc: 0.4587
Epoch 235/500
40000/40000 [==============================] - 23s 566us/step - loss: 0.9746 - acc: 0.6764 - val_loss: 2.4245 - val_acc: 0.4633
Epoch 236/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.8982 - acc: 0.6966 - val_loss: 2.3554 - val_acc: 0.4663
Epoch 237/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9099 - acc: 0.6925 - val_loss: 2.4243 - val_acc: 0.4566
Epoch 238/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0599 - acc: 0.6538 - val_loss: 2.3045 - val_acc: 0.4543
Epoch 239/500
40000/40000 [==============================] - 23s 569us/step - loss: 0.9319 - acc: 0.6873 - val_loss: 2.2857 - val_acc: 0.4456
Epoch 

Epoch 291/500
40000/40000 [==============================] - 23s 572us/step - loss: 0.9423 - acc: 0.6879 - val_loss: 2.5086 - val_acc: 0.4543
Epoch 292/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9702 - acc: 0.6827 - val_loss: 2.4136 - val_acc: 0.4512
Epoch 293/500
40000/40000 [==============================] - 23s 567us/step - loss: 0.9931 - acc: 0.6711 - val_loss: 2.3230 - val_acc: 0.4488
Epoch 294/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9643 - acc: 0.6799 - val_loss: 2.2821 - val_acc: 0.4592
Epoch 295/500
40000/40000 [==============================] - 23s 569us/step - loss: 0.9393 - acc: 0.6878 - val_loss: 2.3343 - val_acc: 0.4592
Epoch 296/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9037 - acc: 0.6987 - val_loss: 2.5753 - val_acc: 0.4574
Epoch 297/500
40000/40000 [==============================] - 23s 571us/step - loss: 0.9579 - acc: 0.6839 - val_loss: 2.5380 - val_acc: 0.4608
Epoch 

Epoch 349/500
40000/40000 [==============================] - 23s 570us/step - loss: 1.0254 - acc: 0.6678 - val_loss: 2.4798 - val_acc: 0.4495
Epoch 350/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0457 - acc: 0.6663 - val_loss: 2.1388 - val_acc: 0.4518
Epoch 351/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0127 - acc: 0.6705 - val_loss: 2.3573 - val_acc: 0.4572
Epoch 352/500
40000/40000 [==============================] - 23s 573us/step - loss: 0.9325 - acc: 0.6912 - val_loss: 2.3750 - val_acc: 0.4505
Epoch 353/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.0785 - acc: 0.6508 - val_loss: 2.3567 - val_acc: 0.4536
Epoch 354/500
40000/40000 [==============================] - 23s 568us/step - loss: 1.0356 - acc: 0.6645 - val_loss: 2.2976 - val_acc: 0.4447
Epoch 355/500
40000/40000 [==============================] - 23s 573us/step - loss: 1.0028 - acc: 0.6688 - val_loss: 2.3985 - val_acc: 0.4540
Epoch 

Epoch 407/500
40000/40000 [==============================] - 23s 571us/step - loss: 1.0324 - acc: 0.6704 - val_loss: 2.6016 - val_acc: 0.4491
Epoch 408/500
40000/40000 [==============================] - 23s 568us/step - loss: 0.9325 - acc: 0.6952 - val_loss: 2.6385 - val_acc: 0.4573
Epoch 409/500
40000/40000 [==============================] - 23s 569us/step - loss: 0.9851 - acc: 0.6791 - val_loss: 2.4675 - val_acc: 0.4484
Epoch 410/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.1227 - acc: 0.6429 - val_loss: 2.3723 - val_acc: 0.4556
Epoch 411/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.0317 - acc: 0.6678 - val_loss: 2.4532 - val_acc: 0.4398
Epoch 412/500
40000/40000 [==============================] - 23s 570us/step - loss: 1.0640 - acc: 0.6572 - val_loss: 2.4301 - val_acc: 0.4586
Epoch 413/500
40000/40000 [==============================] - 23s 567us/step - loss: 1.0305 - acc: 0.6656 - val_loss: 2.3791 - val_acc: 0.4553
Epoch 

Epoch 465/500
40000/40000 [==============================] - 23s 571us/step - loss: 1.0156 - acc: 0.6736 - val_loss: 2.5631 - val_acc: 0.4492
Epoch 466/500
40000/40000 [==============================] - 23s 570us/step - loss: 1.0329 - acc: 0.6662 - val_loss: 2.4756 - val_acc: 0.4534
Epoch 467/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.2788 - acc: 0.6033 - val_loss: 2.0320 - val_acc: 0.4338
Epoch 468/500
40000/40000 [==============================] - 23s 570us/step - loss: 1.1227 - acc: 0.6365 - val_loss: 2.4957 - val_acc: 0.4490
Epoch 469/500
40000/40000 [==============================] - 23s 570us/step - loss: 1.0301 - acc: 0.6667 - val_loss: 2.5388 - val_acc: 0.4592
Epoch 470/500
40000/40000 [==============================] - 23s 577us/step - loss: 1.0121 - acc: 0.6745 - val_loss: 2.5162 - val_acc: 0.4547
Epoch 471/500
40000/40000 [==============================] - 23s 569us/step - loss: 1.0250 - acc: 0.6717 - val_loss: 2.5593 - val_acc: 0.4463
Epoch 

Finally, let's evaluate on our test set:

In [12]:
# Evaluate on validation.
score = model.evaluate(x=x_valid, y=y_valid, batch_size=None, verbose=1, sample_weight=None, steps=None)
print('Validation loss:', score[0])
print('Validation accuracy (NORMALIZED):', score[1])

10000/10000 [==============================] - 3s 275us/step
Validation loss: 2.2372118843078614
Validation accuracy (NORMALIZED): 0.4508


-----------
-----------
-----------
# Tensorboard

Tensorboard is a visualization tool for Tensorflow. Among other things, it allows us to monitor the progress of our training, plot metrics per epochs, visualize the architecture's schematics. 

Just like for Early Stopping, we will use the [Tensorboard callback](https://keras.io/callbacks/#tensorboard) to log the information about our training. An example of usage, would be:

As your training progresses, Keras will log the metrics (e.g., loss, accuracy) to `<<LOG_DIR>>` (**make sure `<<LOG_DIR>>` is a valid directory)**. On your terminal, you will need to run Tensorboard, assign a port and access it via browser (just like jupyter).

#### ----> MAKE SURE YOU USE A DIFFERENT PORT FOR JUPYTER AND TENSORBOARD <----

### Docker
For those using docker, open a new terminal and create a new container (using the same image) running Tensorboard:

For example:

After starting Tensorboard, access it via browser on `http://localhost:<<port_container>>`.

### Anaconda
$ tensorboard --logdir=<<LOG_DIR>> --port=<<port>>

After starting Tensorboard, access it via browser on `http://localhost:<<port>>`.

-----------
-----------
-----------

# Fine-tuning all layers

What if we fine-tune all layers of SqueezeNet?
<img src="unfrozenSqueezeNet.png" width=70% height=70%>

In [13]:
squeezeNetModel = SqueezeNet((32,32,3))

for layer in squeezeNetModel.layers:
    layer.trainable = True       #by default they are all trainable, but just for clarification

#Add new classification layers
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()
squeezeNetModel.layers.pop()

#squeezeNetModel.summary()

x = squeezeNetModel.layers[-1].output
x = Convolution2D(10, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

Now, we compile our model and train it:

In [14]:
#Compile model and train it
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tbCallBack = TensorBoard(log_dir="./TransferLearning/all_layers/{}".format(time()), write_graph=True)

model.fit(x=x_train, y=y_train, batch_size=50, epochs=500, verbose=1, callbacks=[tbCallBack], validation_split=0, 
        validation_data=(x_valid, y_valid), shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, 
        steps_per_epoch=None, validation_steps=None)

model.save('all_layers.h5')

Train on 40000 samples, validate on 10000 samples
Epoch 1/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3238 - acc: 0.0997 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 2/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 3/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 4/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 5/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 6/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 7/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - 

40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 60/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 61/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 62/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 63/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 64/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 65/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 66/500
40000/40000 [==============

Epoch 118/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 119/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 120/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 121/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 122/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 123/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 124/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 125/500
40000/

40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 177/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 178/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 179/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 180/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 181/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 182/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 183/500
40000/40000 [=======

40000/40000 [==============================] - 55s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 235/500
40000/40000 [==============================] - 55s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 236/500
40000/40000 [==============================] - 58s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 237/500
40000/40000 [==============================] - 61s 2ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 238/500
40000/40000 [==============================] - 69s 2ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 239/500
40000/40000 [==============================] - 59s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 240/500
40000/40000 [==============================] - 53s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 241/500
40000/40000 [=======

40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 293/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 294/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 295/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 296/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 297/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 298/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 299/500
40000/40000 [=======

40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 351/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 352/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 353/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 354/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 355/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 356/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 357/500
40000/40000 [=======

40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 409/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 410/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 411/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 412/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 413/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 414/500
40000/40000 [==============================] - 50s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 415/500
40000/40000 [=======

40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 467/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 468/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 469/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 470/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 471/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 472/500
40000/40000 [==============================] - 51s 1ms/step - loss: 2.3026 - acc: 0.0996 - val_loss: 2.3026 - val_acc: 0.1014
Epoch 473/500
40000/40000 [=======

Finally, let's evaluate on our validation set:

In [15]:
# Evaluate on validation
score = model.evaluate(x=x_valid, y=y_valid, batch_size=None, verbose=1, sample_weight=None, steps=None)
print('Validation loss:', score[0])
print('Validation accuracy (NORMALIZED):', score[1])

10000/10000 [==============================] - 3s 288us/step
Validation loss: 2.3025851249694824
Validation accuracy (NORMALIZED): 0.1014


In [16]:
# Evaluate your best model on test
score = model.evaluate(x=x_test, y=y_test, batch_size=None, verbose=1, sample_weight=None, steps=None)

NameError: name 'x_test' is not defined

## Saving the model
Now that we are working on more complex tasks and our trainings are starting to take more time it is usually a good idea to save the trained model from time to time. [Keras has a lot of ways of saving and loading the model](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model), but in this exercise we will use the simplest of them all: `model.save()`. It saves the architecture, the weights, the choice of loss function/optimizer/metrics and even the current state of the training, so you can resume your training later.

In [ ]:
#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

## Loading a model
Once we have our model trained, we can load it using:

In [ ]:
from keras.models import load_model

del model  # Will delete model, only to check if load_model is working

# returns a compiled model identical to the previous one
model = load_model('my_model.h5')

# evaluate test set again... should give us the same result
score = model.evaluate(x=x_test, y=y_test, batch_size=None, verbose=1, sample_weight=None, steps=None)
print('Test loss:', score[0])
print('Test accuracy (NORMALIZED):', score[1])